ANCE

In [1]:
!apt install libomp-dev
!pip install -U python-terrier
!pip install -U git+https://github.com/terrierteam/pyterrier_ance.git
!pip install -U faiss-cpu

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 1s (232 kB/s)
Selecting previously unselected package libomp5:amd64.
(Reading database ... 155013 files and directories currently installed.)
Preparing to unpack .../libomp5_5.0.1-1_amd64.deb ...
Unpacking libomp5:amd64 (5.0.1-1) ...
Selecting previously unselected package libomp-dev.
Preparing to unpack .../libomp-dev_5.0.1-1_amd64.deb ...
Unpacking libomp-dev (5.0.1-

In [2]:
import os
import pyterrier as pt
import pyterrier_ance
import pandas as pd

if not pt.started():
    pt.init()

terrier-assemblies 5.6  jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.6  jar not found, downloading to /root/.pyterrier...
Done
PyTerrier 0.7.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)


In [3]:
def experiment_br(retrs, topics, qrels, labels):
  return pt.Experiment(retrs, topics, qrels, names=labels, eval_metrics=['map', 'ndcg_cut.10', 'P.10', 'mrt'])

In [4]:
dataset_name = 'irds:vaswani'
index_path = './index_vaswani'
props_path = '/data.properties'
dataset = pt.datasets.get_dataset(dataset_name)

if not os.path.exists(index_path):
    indexer = pt.index.IterDictIndexer(index_path)
    index_ref = indexer.index(dataset.get_corpus_iter(), meta=('docno', 'text'))
else:
    index_ref = pt.IndexRef.of(index_path + props_path)

index = pt.IndexFactory.of(index_ref)
topics = dataset.get_topics()
qrels = dataset.get_qrels()
br = pt.BatchRetrieve(index, wmodel='BM25')

[INFO] [starting] building docstore
[INFO] [starting] http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz
docs_iter:   0%|                                     | 0/11429 [00:00<?, ?doc/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 0.0%| 0.00/2.13M [00:00<?, ?B/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 0.8%| 16.4k/2.13M [00:00<00:16, 129kB/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 2.3%| 49.2k/2.13M [00:00<00:11, 182kB/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 4.2%| 90.1k/2.13M [00:00<00:09, 217kB/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 7.3%| 156k/2.13M [00:00<00:07, 278kB/s] 
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 14.6%| 311k/2.13M [00:00<00:04, 442kB/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 30.1%| 639k/2.13M [00:00<00:01, 754kB/s]

[INFO] [finished] http://ir.dcs.gla.ac.uk/resource

vaswani documents:   0%|          | 0/11429 [00:00<?, ?it/s]

In [5]:
if not os.path.exists('Passage_ANCE_FirstP_Checkpoint.zip'):
  !wget https://webdatamltrainingdiag842.blob.core.windows.net/semistructstore/OpenSource/Passage_ANCE_FirstP_Checkpoint.zip
  !unzip Passage_ANCE_FirstP_Checkpoint.zip

path_checkpoint = '/content/Passage ANCE(FirstP) Checkpoint'
path_index_ance = '/content/index_ance'

if not os.path.exists(path_index_ance):
    indexer_ance = pyterrier_ance.ANCEIndexer(checkpoint_path=path_checkpoint, index_path=path_index_ance, num_docs=11429)
    indexer_ance.index(dataset.get_corpus_iter())

--2021-09-23 21:03:39--  https://webdatamltrainingdiag842.blob.core.windows.net/semistructstore/OpenSource/Passage_ANCE_FirstP_Checkpoint.zip
Resolving webdatamltrainingdiag842.blob.core.windows.net (webdatamltrainingdiag842.blob.core.windows.net)... 52.239.193.68
Connecting to webdatamltrainingdiag842.blob.core.windows.net (webdatamltrainingdiag842.blob.core.windows.net)|52.239.193.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1277112820 (1.2G) [application/octet-stream]
Saving to: ‘Passage_ANCE_FirstP_Checkpoint.zip’

Passage_ANCE_FirstP 100%[===================>]   1.19G  18.2MB/s    in 63s     

2021-09-23 21:04:43 (19.2 MB/s) - ‘Passage_ANCE_FirstP_Checkpoint.zip’ saved [1277112820/1277112820]

Archive:  Passage_ANCE_FirstP_Checkpoint.zip
   creating: Passage ANCE(FirstP) Checkpoint/
  inflating: Passage ANCE(FirstP) Checkpoint/config.json  
  inflating: Passage ANCE(FirstP) Checkpoint/desktop.ini  
  inflating: Passage ANCE(FirstP) Checkpoint/merges

vaswani documents:   0%|          | 0/11429 [00:00<?, ?it/s]

[INFO] loading configuration file /content/Passage ANCE(FirstP) Checkpoint/config.json
[INFO] Model config {
  "_num_labels": 2,
  "architectures": [
    "RobertaDot_NLL_LN"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": 0,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": 2,
  "eos_token_ids": 0,
  "finetuning_task": "MSMarco",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-05,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 514,
  "min_length": 0,
  "model_type": "roberta",
  "no_repeat_ngram_size": 0,
  "num_attention_heads": 12,
  "num_beams": 1,
  "num_hidden_layers": 12,
  "num_labels": 2,


Using mean: False


[INFO] Inference parameters <pyterrier_ance. object at 0x7fc90baf5d90>


...

Segment 0


[INFO] ***** Running ANN Embedding Inference *****
[INFO]   Batch size = 128


Inferencing: 0it [00:00, ?it/s]

Not running in distributed mode


[INFO] merging embeddings


In [6]:
ance_retr = pyterrier_ance.ANCERetrieval(checkpoint_path=path_checkpoint, index_path=path_index_ance)
ance_rerank = (br % 100) >> pt.text.get_text(dataset, 'text') >> pyterrier_ance.ANCETextScorer(checkpoint_path=path_checkpoint)

[INFO] loading configuration file /content/Passage ANCE(FirstP) Checkpoint/config.json
[INFO] Model config {
  "_num_labels": 2,
  "architectures": [
    "RobertaDot_NLL_LN"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": 0,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": 2,
  "eos_token_ids": 0,
  "finetuning_task": "MSMarco",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-05,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 514,
  "min_length": 0,
  "model_type": "roberta",
  "no_repeat_ngram_size": 0,
  "num_attention_heads": 12,
  "num_beams": 1,
  "num_hidden_layers": 12,
  "num_labels": 2,


Loading model
Using mean: False


[INFO] loading weights file /content/Passage ANCE(FirstP) Checkpoint/pytorch_model.bin


Loading shard metadata


[INFO] Inference parameters <pyterrier_ance. object at 0x7fc90a96fa10>


Loading shards:   0%|          | 0/1 [00:00<?, ?shard/s]

[INFO] loading configuration file /content/Passage ANCE(FirstP) Checkpoint/config.json
[INFO] Model config {
  "_num_labels": 2,
  "architectures": [
    "RobertaDot_NLL_LN"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": 0,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": 2,
  "eos_token_ids": 0,
  "finetuning_task": "MSMarco",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-05,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 514,
  "min_length": 0,
  "model_type": "roberta",
  "no_repeat_ngram_size": 0,
  "num_attention_heads": 12,
  "num_beams": 1,
  "num_hidden_layers": 12,
  "num_labels": 2,


Using mean: False


[INFO] Inference parameters <pyterrier_ance. object at 0x7fc90b93b510>


In [7]:
experiment_br([br, ance_retr, ance_rerank], topics, qrels, ['BM25', 'ANCE', 'BM25 >> ANCE'])

***** inference of 93 queries *****


[INFO] ***** Running ANN Embedding Inference *****
[INFO]   Batch size = 128


Inferencing: 0it [00:00, ?it/s]

Not running in distributed mode
***** faiss search for 93 queries on 1 shards *****


[INFO] merging embeddings


...

[INFO] merging embeddings
[INFO] ***** Running ANN Embedding Inference *****
[INFO]   Batch size = 128


Inferencing: 0it [00:00, ?it/s]

Not running in distributed mode


[INFO] merging embeddings


,name,map,ndcg_cut.10,P.10,mrt
0,BM25,0.296517,0.446609,0.352688,84.187963
1,ANCE,0.163872,0.342211,0.263441,175.524590
2,BM25 >> ANCE,0.228010,0.392498,0.317204,35602.744920


end of fun.